In [ ]:
#!/usr/bin/env python
# coding: utf-8

"""
Descriptor Convolutional Neural Network
"""

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Install packages

In [ ]:
! pip install nvgpu
! pip install py-cpuinfo
! pip install timer
! python -m pip install psutil==5.7.2 --user

# Check if is running on Colab

In [ ]:
use_colab = 'google.colab' in str(get_ipython())

if use_colab:
    # Define base path
    BASE_PATH = '/content/gdrive/My Drive/Colab Notebooks/DESC-NET/src/'

    # Import Libraries from colab
    from google.colab import drive
    from google.colab import output
    import IPython
    
    # Mount Google Drive for fast, responsible access to files
    drive.mount('/content/gdrive')
    
    message_use_colab = '[INFO] Running on Colab:\n'
else:
    message_use_colab = '[INFO] Not running on Colab:\n'
    pass

# Colab Runtime to prevent from disconnecting

In [ ]:
if use_colab:
    display(IPython.display.Javascript('''
    function ClickConnect(){
        btn = document.querySelector("colab-connect-button")
        if (btn != null){
          console.log("Click colab-connect-button"); 
          btn.click() 
        }
        
        btn = document.getElementById('ok')
        if (btn != null){
          console.log("Click reconnect"); 
          btn.click() 
        }
      }
        
    setInterval(ClickConnect, 60000)'''))
else:
    pass

# Import Libraries

In [ ]:
from PIL import Image
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import TopKCategoricalAccuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2 as cv
import glob
import json
import numpy as np
import os
import pickle
import random
import tensorflow as tf
import time
import timeit
import zipfile

# Import Custom Libraries

In [ ]:
# To import and use the custom imports, the folder need
# to be in se same directory to de file (in colab) 
if use_colab:
    # Unzip in Colab Workspace
    UTILITIES = 'utilities.zip'

    UTILITIES_PATH =  "/".join([BASE_PATH, UTILITIES])

    zip_object = zipfile.ZipFile(file = UTILITIES_PATH,
                                mode = 'r')

    zip_object.extractall()

    zip_object.close

from utilities.dataset import(#manipulate_pandas,
                              create_dataset
)

from utilities.descriptors import find_detector_or_descriptor

from utilities import global_variables

from utilities.logfile import(close_logfile,
                              open_logfile,
                              print_descriptor_initialized_info,
                              print_detector_initialized_info,
                              print_system_info,
                              print_packages_info
)

from utilities.timer import Timer

from utilities.visualizations import(#save_dataset_distribution,
                                     save_architecture_visualization,
                                     save_model_accuracy_history,
                                     save_model_loss_history,
                                     save_confusion_matrix,
)


from utilities.local_feature_detection import local_feature_detection

from utilities.local_descriptor_convolution import * # TODO: Arrumar e importar todos... depois...

from utilities.residual_neural_network import ResNet

from utilities.model_functions import(training,
                                      evaluate,
                                      predict)

# Import config.json
## Used to load the user configurations

In [ ]:
if use_colab:
    # Unzip in Colab Workspace
    JSON = 'config-FEI.zip'

    JSON_PATH =  "/".join([BASE_PATH, JSON])

    zip_object = zipfile.ZipFile(file = JSON_PATH,
                                 mode = 'r')

    zip_object.extractall()

    zip_object.close 

JSON_PATH = 'config.json'

# Load the user configurations
with open(JSON_PATH, encoding='utf8') as f:    
    config = json.load(f)

# Config variables

In [ ]:
# Stores the dataset's name
DATASET = config['dataset']

try:
    assert(DATASET == 'MNIST' or
           DATASET == 'JAFFE' or
           DATASET == 'Extended-CK+' or
           DATASET == 'CIFAR-10' or
           DATASET == 'CIFAR-100' or
           DATASET == 'FEI' or
           DATASET == 'FER-2013' or
           DATASET == 'ILSVRC-2017')
except AssertionError as e:
    raise(AssertionError('`DATASET` cannot be different from MNIST or JAFFE or Extended-CK+ or CIFAR-10 or CIFAR-100 or FEI or FER-2013 or ILSVRC-2017'))

# Path of training-set
DATA_TRAIN_IMAGES = "/".join([config['dataset'], config['data_train_images']])

# Path of test-set
DATA_TEST_IMAGES = "/".join([config['dataset'], config['data_test_images']])

# Stores the detector's name
DETECTOR = config['detector']

try:
    assert(DETECTOR == 'SIFT' or
           DETECTOR == 'SURF' or
           DETECTOR == 'KAZE' or
           DETECTOR == 'ORB' or
           DETECTOR == 'BRISK' or
           DETECTOR == 'AKAZE')
except AssertionError as e:
    raise(AssertionError('`DETECTOR` cannot be different from SIFT or SURF or KAZE or ORB or BRISK or AKAZE'))

# Stores the descriptor's name
DESCRIPTOR = config['descriptor']

try:
    assert(DESCRIPTOR == 'SIFT' or
           DESCRIPTOR == 'SURF' or
           DESCRIPTOR == 'KAZE' or
           DESCRIPTOR == 'BRIEF' or
           DESCRIPTOR == 'ORB' or
           DESCRIPTOR == 'BRISK' or
           DESCRIPTOR == 'AKAZE' or
           DESCRIPTOR == 'FREAK')
except AssertionError as e:
    raise(AssertionError('`DESCRIPTOR` cannot be different from SIFT or SURF or KAZE or BRIEF or ORB or BRISK or AKAZE or FREAK'))

# Stores the reducer's name
#REDUCER = config['reducer']

# Stores the architecture's name of CNN
ARCHITECTURE = config['architecture']

try:
    assert(ARCHITECTURE == 'DescNet-50' or
           ARCHITECTURE == 'DescNet-101' or
           ARCHITECTURE == 'DescNet-152' or
           ARCHITECTURE == 'ResNet-50' or
           ARCHITECTURE == 'ResNet-101' or
           ARCHITECTURE == 'ResNet-152')
except AssertionError as e:
    raise(AssertionError('`ARCHITECTURE` cannot be different from DescNet-50 or DescNet-101 or DescNet-152 or ResNet-50 or ResNet-101 or ResNet-152'))

# Store's desc block if used
USE_DESC_BLOCK = config['use_desc_block']

# Define base path
if not use_colab:
    BASE_PATH = config['main_path']
    

# Path
DESCRIPTOR_PATH = "/".join([config['dataset'], config['architecture'], config['descriptor']])

# Where models are stored
MODEL_PATH = "/".join([BASE_PATH, 'Models', DESCRIPTOR_PATH])

# Stores model data and checkpoints
CHECKPOINT_PATH = "/".join([BASE_PATH, 'Pretrained', DESCRIPTOR_PATH])

# Initializing global variables

In [ ]:
global_variables.initialize()

# Simple line to difide dome steps of this project
global_variables.LINE

# Absolute output path
global_variables.OUTPUT_PATH = "/".join([BASE_PATH, 'Outputs', DESCRIPTOR_PATH])

print(global_variables.OUTPUT_PATH)

# Unzip outside Dataset

In [ ]:
DATASET_PATH = '/'.join([BASE_PATH, 'Datasets', DATASET + '.zip'])

zip_object = zipfile.ZipFile(file = DATASET_PATH,
                             mode = 'r')

zip_object.extractall('./')

zip_object.close

# Model Files

In [ ]:
# if true, get the last one
if len(os.listdir(CHECKPOINT_PATH)) != 0:
        # List of files
        list_of_files = os.listdir(CHECKPOINT_PATH)

        list = []
        for file in list_of_files:
            list.append('/'.join([CHECKPOINT_PATH, file]))

        # Get the last one checkpoint to load and continue from
        h5_file = max(list, key = os.path.getmtime)
        print(h5_file)

json_file = '/'.join([MODEL_PATH, config['architecture'] + '.json'])

In [ ]:
json_file = open(json_file, 'r')

loaded_model_json = json_file.read()

json_file.close()

In [ ]:
with CustomObjectScope({'DescConv2D': DescConv2D}):
    loaded_model = tf.keras.models.load_model(h5_file)

## Carregamento dos dados para gerar a matriz de confusão

## Gerando a Matriz de Confusão

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

# Classes info
CLASSES = os.listdir(DATA_TRAIN_IMAGES)

#load 
y_true = np.load(global_variables.OUTPUT_PATH + '/y_true.npy')
y_pred = np.load(global_variables.OUTPUT_PATH + '/y_pred.npy')

# Get the confusion matrix using sklearn
cm = confusion_matrix(y_true = y_true, y_pred = y_pred)

# Print the confusion matrix as text
# print(cm)

# Plot the confusion matrix as an image
thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j,
             i,
             cm[i, j],
             horizontalalignment='center',
             color='white' if cm[i, j] > thresh else 'black')


# Make various adjustments to the plot
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.colorbar()
tick_marks = np.arange(len(CLASSES))
plt.xticks(tick_marks, CLASSES, rotation = 45)
plt.yticks(tick_marks, CLASSES)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig(global_variables.OUTPUT_PATH + '/matriz_confusao_mod_alt.png')
plt.show()